<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import re
import numpy as np
import pandas as pd
import gc

import functools as fnc
import operator as opr

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,LancasterStemmer

from pymystem3 import Mystem 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier,Pool
from catboost.text_processing import Tokenizer

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

import matplotlib.pyplot as plt

## Подготовка

In [2]:
np.random.seed(499)

### Загрузка стопслов из NLTK

In [3]:
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('english')))
stopwords

[nltk_data] Downloading package stopwords to /home/ltz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['nor',
 'does',
 'below',
 'theirs',
 'are',
 'you',
 'at',
 'again',
 'needn',
 'over',
 "you'll",
 'wasn',
 'up',
 'other',
 'had',
 'here',
 'few',
 'having',
 's',
 'shan',
 'any',
 "wasn't",
 'but',
 'doing',
 'was',
 'aren',
 'his',
 'how',
 'we',
 "didn't",
 "wouldn't",
 'between',
 'did',
 'me',
 'too',
 'mustn',
 'these',
 'an',
 'hasn',
 "that'll",
 'll',
 "should've",
 'he',
 'only',
 'o',
 'through',
 'i',
 'in',
 'him',
 'has',
 'then',
 'can',
 'were',
 'for',
 'under',
 'be',
 'who',
 'out',
 'our',
 'no',
 'hers',
 "it's",
 'y',
 'am',
 'of',
 'himself',
 'is',
 'that',
 'myself',
 'been',
 'those',
 'with',
 "mustn't",
 'now',
 'ourselves',
 "don't",
 "you're",
 'and',
 'until',
 "couldn't",
 'same',
 'weren',
 'they',
 'their',
 'if',
 'by',
 'or',
 'while',
 'them',
 'will',
 'wouldn',
 "haven't",
 'my',
 'themselves',
 'during',
 're',
 'into',
 'her',
 'it',
 'on',
 'about',
 'your',
 'because',
 'herself',
 "shan't",
 'when',
 'do',
 'don',
 'as',
 'ma',
 'should

### Загрузка данных

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv',index_col=0,encoding='utf-8')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Лемматизация 

In [48]:
%%time
mst = Mystem()
lst = LancasterStemmer()
snb = SnowballStemmer('english')
stemmer = lst.stem

def lemmatize(text,stemmer):
    pure = re.sub(r"([^a-z\'])+",' ',text.lower())
    words =  pure.split()  
    return( list( map(stemmer,words)) )
#     print(text)
#     text = re.sub('( )+',' ',text)
#    return ''.join( stemmer(text)).strip('\n')                
def lanc_stem(text):
    return lemmatize(text,lst.stem)
def snow_stem(text):
    return lemmatize(text,snb.stem)

def my_stem(text):
    return mst.lemmatize(text)
# lanc_stem.__name__='lancaster'
# my_stem.__name__ = 'mystem'

print( lanc_stem('a  lot of   matches') )
print( snow_stem('a  lot of   matches') )
print( my_stem('matches') )
# lemmatize(df.text[2])                    
#df['lemmas'] = df.text.apply(lemmatize)

# def lemmatizer(serie):
#     return serie.apply(lemmatize)
# #df.sample(10).lemmas.apply(lambda st: print(f" {st:.99s}") )

# lemmatizer(df.text.head(20))

# TfidfVectorizer(tokenizer=lemmatize).fit_transform(df.text.head(20)).todense()

Exception ignored in: <function Mystem.__del__ at 0x7f75ac277d30>
Traceback (most recent call last):
  File "/home/ltz/anaconda3/lib/python3.9/site-packages/pymystem3/mystem.py", line 196, in __del__
    self.close()  # terminate process on exit
  File "/home/ltz/anaconda3/lib/python3.9/site-packages/pymystem3/mystem.py", line 216, in close
    if self._proc is not None:
AttributeError: 'Mystem' object has no attribute '_proc'


TypeError: __init__() got an unexpected keyword argument 'language'

#### Разбиение на тренировочный и тестовый наборы

In [7]:
%%time
X = 'text'  
y = 'toxic'

tr,te = train_test_split(df, test_size = .25, shuffle = True, stratify=df[y])



CPU times: user 68.8 ms, sys: 0 ns, total: 68.8 ms
Wall time: 66.1 ms


## Обучение

#### Вспомогательные классы

In [49]:
gs.cv_results_

{'mean_fit_time': array([38.17715627, 30.66127861, 35.12304491, 28.29321259, 36.44178987,
        29.09740239, 35.34423548, 28.3945964 ]),
 'std_fit_time': array([0.52054316, 1.43671587, 0.27165651, 0.11460409, 0.44121281,
        0.26119635, 0.14528297, 0.16966969]),
 'mean_score_time': array([12.71362698,  9.60080045, 11.492836  ,  9.17770272, 11.8043955 ,
         9.32051241, 11.59718955,  9.16020846]),
 'std_score_time': array([0.58186459, 0.52616134, 0.18117816, 0.06713648, 0.20182173,
        0.11155441, 0.14544406, 0.08977537]),
 'param_model__solver': masked_array(data=['lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'lbfgs',
                    'lbfgs', 'lbfgs'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_vect__max_features': masked_array(data=[3000, 3000, 3000, 3000, 4000, 4000, 4000, 4000],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=

##### Классы для преобразваний результата кроссвалидации в датафрейм

In [22]:
class ParsedResult:
    def __init__(self,result,metric_name='score',sample_name='mean'):
        self.df = pd.DataFrame.from_dict( 
            { k:v for k,v in result.items() if k not in ['params']}
        )
# drop param_ prefix from column names for tidy display         
        self.primary_score=f"{sample_name}_test_{metric_name}"
        param_names = [ c for c in self.df.columns if c[:6]=='param_' ]        
        self.param_cols = [ c[6:] for c in param_names ]              
        self.df = self.df.rename( columns={ p:c for p,c in zip(param_names,self.param_cols) } )
# also drop model_ prefix which we use for model grid in pipeline   
        model_names = [ c for c in self.df.columns if c[:7]=='model__' ]
        self.param_cols = [c[7:]  if c[:7]=='model__' else c for c in self.param_cols ]
        self.model_cols = [ c[7:] for c in model_names ]   
        self.df = self.df.rename( columns={ p:c for p,c in zip(model_names,self.model_cols) } )
# some object columns may be not string (f.e. lists)
        for i in range(len(self.df.columns)):
            if self.df.dtypes[i]=='object':
                self.df[self.df.columns[i]] = self.df[self.df.columns[i]].astype(str)
# select parameters, for which tested several values and arrange them descending (by number of differnt values)
        values_per_cols = list(zip( self.param_cols,[ len( np.unique( self.df[c].values ) ) for c in self.param_cols]))
        self.multi_value_cols =[it[0] for it in sorted(values_per_cols, key=lambda pair:pair[1],reverse=True) if it[1]>1 ]

        
            
    def select(self, index,values=[],filters={},agg=[] ):
        values = [ [self.primary_score] ,values][bool(values)]
        if len(filters)>0 :
            condition = 'and'.join([ f" {k}=={v} " for k,v in filters.items() ]) 
            filtered = self.df.query(condition)    
        else:
            filtered = self.df
        cols = [ c for c in self.multi_value_cols if c not in list(filters.keys())+[index]+values ]      
        return(self.df.pivot_table(index=index,values=values,columns=cols)  )
     

##### Классы для презентации графиков

In [18]:
class Styler:
    base_array=[]
    def __init__(self,param_array=[]):
        self.params = [ self.base_array,param_array][bool(param_array)]
    
    def key(self):
        pass
    
    def put(self,dct,index):
        dct[self.key()] = self.params[index]
        
class ColorStyler(Styler):
    base_array=[*'rgbykm']
    def key(self):
        return('color')

class DashStyler(Styler):
    base_array=['--',':','-.','-']
    def key(self):
        return('ls')
    
class WidthStyler(Styler):
    base_array=[ 2,4,7]
    def key(self):
        return('lw')

class AlphasStyler(Styler):
    base_array=[ .3,.9]
    def key(self):
        return('alpha')


In [20]:
class ResultPlotter:
    base_params = {
        'colors':[*'rgbykm'],
        'styles':['--',':','-.','-',(0, (1, 10)),(0, (5, 10))],
        'widths':[ 2,4,7],
        'alphas':[.3,.8],
        'figsize': (20,8),
        'logscale':""
    }
    
    def __init__(self, test, params = {} ):
        self.model = test.model
        self.df= test.sel
        self.params=self.base_params | params
        self.stylers = [ColorStyler(self.params['colors']),DashStyler(self.params['styles']),
                        WidthStyler(self.params['widths']),AlphasStyler(self.params['alphas'])]
        self.mcols = self.df.columns      
        self.lev_names = list(self.mcols.names)
        self.num_levels= len(self.lev_names)
        self.level_sizes=[ len(set(self.mcols.get_level_values(x))) for x in self.lev_names]
        
        # if level 0("values of pivot") is one-valued, then start apply styles from columns 1  
        self.start_level = int(  self.level_sizes[0] == 1   )
# TODO: refactoring needed here, one or two new functions to encapsulate this feature
#       also to print short description for " result thread" 

        

# auxliary func to forms sequence of indexes for levels
# result is need index on the level  which may be one of color|dashstyle|linewidth
    def idx_on_level(self,idx, lev_num):
        items_on_hyperplane = multiply(self.level_sizes[(lev_num+1):]) 
        return idx//items_on_hyperplane% self.level_sizes[lev_num]
    
    def plot(self):
        fig,ax = plt.subplots(figsize=(20,6))
        if 'x' in self.params['logscale'].lower():
            ax.set_xscale('log')
        if not self.lev_names[0]:
            self.lev_names[0]='res.thread'
            
        for i_mcol in range(len( self.mcols)):
            graph_params = {}
            for i_level in range(self.start_level,self.num_levels):
                self.stylers[i_level-self.start_level].put( graph_params, self.idx_on_level(i_mcol,i_level) ) 
            idx=self.mcols[i_mcol]
            
            levels_for_label=[ f"{n}:{j} " for n,j in zip(self.lev_names[self.start_level:],idx[self.start_level:]) ] 
            graph_params['label'] = ','.join( levels_for_label )
            ax.plot(self.df[idx] ,**graph_params)
            ax.set_title(f"Тест {self.params['title']}")
            ax.set_xlabel('learning rate')
            ax.set_ylabel(f"{self.params['metric']}")
            ax.legend()

#### Подбор параметров словаря

 В принципе, для пользовния языком достаточно 4-5К слов , больший активный словарь уже признак хорошего литературного языка и такой объем вряд ли используется в коротких текстах.    
 Понятно, что чем больше словарь  -  тем точнее, но тест  ограничен ресурсами компьютера и временем, так что я хочу подобрать такой размер словаря , расширение которого  уже не дает существенного улучшения результата.   
 Я предполагаю, что потеря метрики от словаря мало зависит от модели , поэтому для скорости использую LogisticRegression.   
 Здесь же проверяется эффект от добавления в словарь n-грамм

In [46]:
%%time
pipe = Pipeline(steps = [
    ['vect',  TfidfVectorizer(tokenizer=lemmatize,stop_words = stopwords)],
    ['model', LogisticRegression(max_iter= 2000)]
])
params_grid = {
    'model__solver': ['lbfgs'],
    'vect__max_features':[3_000,4_000],#,5000,6_000,7_000,8_000,9_000],
#    'vect__min_df' : [1/5_000,1/10_000],
    'vect__tokenizer': [lanc_stem,snow_stem],
    'vect__stop_words': [ [' '],stopwords],    
    'vect__ngram_range':[(1,1)]#,(1,2),(1,3),(1,4)]
}
gs=GridSearchCV(pipe, params_grid,
                cv=4 ,
                scoring = 'f1')
gs.fit(tr[X].values,tr[y])
gs.cv_results_

/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/

/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'aft', 'al', 'ar', 'becaus', 'bef', 'doe', 'dur', 'furth', 'hav', 'mor', 'ont', 'oth', 'ourselv', 'ov', 'sam', 'som', 'themselv', 'ther', 'thes', 'thi', 'thos', 'und', 'wer', 'wher', 'whil', 'wil', 'yo', 'yourselv'] not in stop_words.
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with 

CPU times: user 23min 53s, sys: 949 ms, total: 23min 54s
Wall time: 23min 54s


{'mean_fit_time': array([38.17715627, 30.66127861, 35.12304491, 28.29321259, 36.44178987,
        29.09740239, 35.34423548, 28.3945964 ]),
 'std_fit_time': array([0.52054316, 1.43671587, 0.27165651, 0.11460409, 0.44121281,
        0.26119635, 0.14528297, 0.16966969]),
 'mean_score_time': array([12.71362698,  9.60080045, 11.492836  ,  9.17770272, 11.8043955 ,
         9.32051241, 11.59718955,  9.16020846]),
 'std_score_time': array([0.58186459, 0.52616134, 0.18117816, 0.06713648, 0.20182173,
        0.11155441, 0.14544406, 0.08977537]),
 'param_model__solver': masked_array(data=['lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'lbfgs',
                    'lbfgs', 'lbfgs'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_vect__max_features': masked_array(data=[3000, 3000, 3000, 3000, 4000, 4000, 4000, 4000],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=

In [50]:

multiply = lambda array: fnc.reduce( opr.mul,array,1 )
p = ParsedResult(gs.cv_results_)
sel = p.select('vect__max_features')
sel

mean_test_score  \
vect__stop_words                                    [' ']   
vect__tokenizer    <function lanc_stem at 0x7f7572be7ca0>   
vect__max_features                                          
3000                                             0.748437   
4000                                             0.749844   

                                                           \
vect__stop_words                                            
vect__tokenizer    <function snow_stem at 0x7f7572b61820>   
vect__max_features                                          
3000                                             0.748421   
4000                                             0.749637   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      \
vect__stop_words   ['nor', 'does', 'below', 'theirs', 'are', 'you', 'at', 'again', 'needn', 'over', "you'll", 'wasn', 'up', 'other', 'had', 'here', 'few', 'having', 's', 'shan', 'any', "wasn't", 'but', 'doing', 'was', 'aren', 'his', 'how', 'we', "didn't", "wouldn't", 'between', 'did', 'me', 'too', 'mustn', 'these', 'an', 'hasn', "that'll", 'll', "should've", 'he', 'only', 'o', 'through', 'i', 'in', 'him', 'has', 'then', 'can', 'were', 'for', 'under', 'be', 'who', 'out', 'our', 'no', 'hers', "it's", 'y', 'am', 'of', 'himself', 'is', 'that', 'myself', 'been', 'those', 'with', "mustn't", 'now', 'ourselves', "don't", "you're", 'and', 'until', "couldn't", 'same', 'weren', 'they', 'their', 'if', 'by', 'or', 'while', 'them', 'will', 'wouldn', "haven't", 'my', 'themselves', 'during', 're', 'into', 'her', 'it', 'on', 'about', 'your', 'because', 'herself', "shan't", 'when', 'do', 'don', 'as', 'ma', 'should', 'just', 'further', "you'd", 'than', 'haven', 'from', 'yourself', 've', 'its', 'most', 'both', 'she', 'own', 'all', 'once', 'why', "hasn't", "weren't", 'being', 'mightn', 'each', 'whom', 'not', 'yourselves', 'ain', 'the', 'yours', 'what', 't', 'hadn', 'a', 'after', 'this', 'd', 'doesn', "won't", 'won', "hadn't", "you've", 'some', "isn't", 'such', 'which', 'didn', 'above', "shouldn't", 'before', "she's", "doesn't", 'shouldn', "needn't", 'more', 'ours', 'to', "aren't", 'itself', 'where', 'couldn', 'there', 'against', 'm', "mightn't", 'very', 'have', 'down', 'off', 'isn', 'so']   
vect__tokenizer                                                                                                                                                                                                                                                                                                             

In [24]:
ResultPlotter(sel).plot()

AttributeError: 'DataFrame' object has no attribute 'model'

Расширение словаря за 6000 слов не дает существенного прироста метрики,
Несколько неожиданно , что учёт сочетаний слов ухудшает метрику, и чем длиннее сочетания, тем хуже.
Дальше вся работа идет со словарем 7_000 слов и  n-граммами (1,1)

#### Подбор параметров модели 

Обсчет модели занимает довольно много времени, слеудющий блок выполняется почти 3 часа.  
Для ускорения расчет производится на субсэмплах.
Подбирается оптимальная скорость обучения и способ балансировки для 1024 оценщиков.  

In [ ]:
%%time
pipe = Pipeline(steps = [
    ['vect',  TfidfVectorizer(stop_words = stopwords, max_features=7_000, ngram_range=(1,1))],
    ['model', CatBoostClassifier(
                n_estimators = 1024,
                bootstrap_type='Bernoulli',subsample=.1,
                eval_metric='F1',verbose =128) ]
])
params_grid = {    
    'model__learning_rate':[.05,.1,.2,.5],
    'model__auto_class_weights':['None','Balanced','SqrtBalanced']
}
gs=GridSearchCV(pipe, params_grid,
                cv=3 ,
                scoring = 'f1')
gs.fit(tr[X].values,tr[y])
gs.cv_results_

In [ ]:
multiply = lambda array: fnc.reduce( opr.mul,array,1 )
p = ParsedResult(gs.cv_results_)
sel = p.select('learning_rate')
sel

In [ ]:
ResultPlotter(sel).plot()

Для финального теста выбран лучший результат - баланс SqrtBalance, скорость обучения .2,   
  она попадет в целевую зону по результатам кроссвалидации.    
Возможно модель без баланса недостаточно обучена  и из неё можно выжать больше, но проверка обучения займёт слишком много времени 

#### Финальный тест

In [ ]:
tf = TfidfVectorizer(stop_words = stopwords, max_features=7_000, ngram_range=(1,1))
cb = CatBoostClassifier(n_estimators = 1024,
#                bootstrap_type='Bernoulli',subsample=.1,
                    learning_rate=.2,auto_class_weights='SqrtBalanced',
                   random_state=4999,
                   eval_metric='F1',verbose =128)
pipe = Pipeline( [['vect',tf], ['model',cb] ] )
pipe.fit(tr[X],tr[y])

pr = pipe.predict(te[X])
for metric in [f1_score,accuracy_score,precision_score,recall_score]:
    print( f"{metric.__name__}:{round( metric(te[y],pr), 4)}\t" )
confusion_matrix(te[y],pr)

При финальном тесте достигнуто значительное улучшение метрики - <b>0.788</b>  
Причиной улучшение я считаю использованипе полного набора данных при финальном тестировании 

In [ ]:
raise

## Выводы

## Чек-лист проверки